In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import time
import os
import argparse
## Load the model 
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

In [2]:
model_conv = torchvision.models.inception_v3(pretrained='imagenet')

In [3]:

def mixup_batch(inp_batch, alpha):
    """
    Applies mixup augementation to a batch
    :param input_batch: tensor with batchsize as first dim
    :param alpha: lamda drawn from beta(alpha+1, alpha)
    """
    inp_clone = inp_batch.clone()
    #getting batch size
    batchsize = inp_batch.size()[0]

    #permute a clone
    perm = np.random.permutation(batchsize)
    for i in range(batchsize):
        inp_clone[i] = inp_batch[perm[i]]
    #generating different lambda for each sample
    #Refernced from http://www.inference.vc/mixup-data-dependent-data-augmentation/
    lam = torch.Tensor(np.random.beta(alpha+1, alpha, batchsize))
    lam = lam.view(-1,1,1,1)
    inp_mixup = lam * inp_batch + (1- lam) * inp_clone
    return inp_mixup

In [4]:
ckptpth="models/ckptk.model"

In [5]:
def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, use_gpu, num_epochs=25, mixup = False, alpha = 0.1):
    print("MIXUP".format(mixup))
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0
    i=0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.eval() # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in tqdm(dataloaders[phase]):
                # get the inputs
                inputs, labels = data

                #augementation using mixup
                if phase == 'train' and mixup:
                    inputs = mixup_batch(inputs, alpha)

                if use_gpu:
                    inputs = inputs.cuda()
                    labels = labels.cuda()
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # # forward
                # outputs = model(inputs)
                # if type(outputs) == tuple:
                #     outputs, _ = outputs
                # _, preds = torch.max(outputs.data, 1)
                # loss = criterion(outputs, labels)

                with torch.set_grad_enabled(phase== 'train'):
                    outputs = model(inputs)
                    if type(outputs) == tuple:
                        outputs, _ = outputs
                    _,preds = torch.max(outputs,1)
                    loss=criterion(outputs,labels)
                # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.data.item()
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc >= best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
            #model.load_state_dict(best_model_wts)
                torch.save(model,ckptpth+str(i))
                i=i+1
            
        #print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [6]:
freeze_layers=1
n_class=5

In [7]:
## Lets freeze the first few layers. This is done in two stages 
# Stage-1 Freezing all the layers 
if freeze_layers:
  for i, param in model_conv.named_parameters():
    param.requires_grad = False

# Since imagenet as 1000 classes , We need to change our last layer according to the number of classes we have,
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, n_class)

# Stage-2 , Freeze all the layers till "Conv2d_4a_3*3"
ct = []
for name, child in model_conv.named_children():
    if "Conv2d_4a_3x3" in ct:
        for params in child.parameters():
            params.requires_grad = True
    ct.append(name)
    print(name)
    
# To view which layers are freeze and which layers are not freezed:
for name, child in model_conv.named_children():
    print(name)
    for name_2, params in child.named_parameters():
        print(name_2, params.requires_grad)

Conv2d_1a_3x3
Conv2d_2a_3x3
Conv2d_2b_3x3
Conv2d_3b_1x1
Conv2d_4a_3x3
Mixed_5b
Mixed_5c
Mixed_5d
Mixed_6a
Mixed_6b
Mixed_6c
Mixed_6d
Mixed_6e
AuxLogits
Mixed_7a
Mixed_7b
Mixed_7c
fc
Conv2d_1a_3x3
conv.weight False
bn.weight False
bn.bias False
Conv2d_2a_3x3
conv.weight False
bn.weight False
bn.bias False
Conv2d_2b_3x3
conv.weight False
bn.weight False
bn.bias False
Conv2d_3b_1x1
conv.weight False
bn.weight False
bn.bias False
Conv2d_4a_3x3
conv.weight False
bn.weight False
bn.bias False
Mixed_5b
branch1x1.conv.weight True
branch1x1.bn.weight True
branch1x1.bn.bias True
branch5x5_1.conv.weight True
branch5x5_1.bn.weight True
branch5x5_1.bn.bias True
branch5x5_2.conv.weight True
branch5x5_2.bn.weight True
branch5x5_2.bn.bias True
branch3x3dbl_1.conv.weight True
branch3x3dbl_1.bn.weight True
branch3x3dbl_1.bn.bias True
branch3x3dbl_2.conv.weight True
branch3x3dbl_2.bn.weight True
branch3x3dbl_2.bn.bias True
branch3x3dbl_3.conv.weight True
branch3x3dbl_3.bn.weight True
branch3x3dbl_3.bn.bi

In [15]:
data_dir = "Datas/"

In [16]:
    
## Loading the dataloaders -- Make sure that the data is saved in following way
"""
data/
  - train/
      - class_1 folder/
          - img1.png
          - img2.png
      - class_2 folder/
      .....
      - class_n folder/
  - val/
      - class_1 folder/
      - class_2 folder/
      ......
      - class_n folder/
"""

input_shape = 299
batch_size = 10
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]
scale = 360
input_shape = 299 
use_parallel = True
use_gpu = True
epochs = 10

data_transforms = {
        'train': transforms.Compose([
        transforms.Resize(scale),
        transforms.RandomResizedCrop(input_shape),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(degrees=90),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)]),
        'val': transforms.Compose([
        transforms.Resize(scale),
        transforms.CenterCrop(input_shape),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)]),}

In [17]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                      data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                         shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [18]:
class_names

['aircrafts_1', 'birds_1', 'cars_1', 'dogs_1', 'flowers_1']

In [67]:
for i in dataloaders:
    print(i[0])

v
t


In [34]:
'''if use_parallel:
    print("[Using all the available GPUs]")
    model_conv = nn.DataParallel(model_conv, device_ids=[0, 1])
'''
print("[Using CrossEntropyLoss...]")
criterion = nn.CrossEntropyLoss()

print("[Using small learning rate with momentum...]")
optimizer_conv = optim.SGD(list(filter(lambda p: p.requires_grad, model_conv.parameters())), lr=0.001, momentum=0.9)

print("[Creating Learning rate scheduler...]")
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

print("[Training the model begun ....]")
#torch.cuda.empty_cache() 
# train_model function is here: https://github.com/Prakashvanapalli/pytorch_classifiers/blob/master/tars/tars_training.py
model_ft = train_model(model_conv.cuda(), dataloaders, dataset_sizes, criterion, optimizer_conv, exp_lr_scheduler, 1,
                     num_epochs=epochs)





  0%|          | 0/142 [00:00<?, ?it/s]

[Using CrossEntropyLoss...]
[Using small learning rate with momentum...]
[Creating Learning rate scheduler...]
[Training the model begun ....]
MIXUP
Epoch 0/9
----------






  1%|          | 1/142 [00:01<04:38,  1.98s/it]



  1%|▏         | 2/142 [00:02<03:55,  1.68s/it]



  2%|▏         | 3/142 [00:04<03:30,  1.52s/it]



  3%|▎         | 4/142 [00:05<03:08,  1.37s/it]



  4%|▎         | 5/142 [00:06<02:53,  1.26s/it]



  4%|▍         | 6/142 [00:07<02:43,  1.20s/it]



  5%|▍         | 7/142 [00:08<02:38,  1.18s/it]



  6%|▌         | 8/142 [00:09<02:31,  1.13s/it]



  6%|▋         | 9/142 [00:10<02:26,  1.10s/it]



  7%|▋         | 10/142 [00:11<02:22,  1.08s/it]



  8%|▊         | 11/142 [00:12<02:20,  1.07s/it]



  8%|▊         | 12/142 [00:13<02:17,  1.06s/it]



  9%|▉         | 13/142 [00:14<02:16,  1.06s/it]



 10%|▉         | 14/142 [00:15<02:15,  1.06s/it]



 11%|█         | 15/142 [00:16<02:15,  1.07s/it]



 11%|█▏        | 16/142 [00:17<02:13,  1.06s/it]



 12%|█▏        | 17/142 [00:18<02:14,  1.07s/it]



 13%|█▎        | 18/142 [00:19<02:11,  1.06s/it]



 13%|█▎        | 19/142 [00:20<02:09,  1.05s/it]



 14%|█▍        | 

train Loss: 0.0179 Acc: 0.9582






  2%|▏         | 1/48 [00:03<02:37,  3.36s/it]



  4%|▍         | 2/48 [00:04<02:04,  2.70s/it]



  6%|▋         | 3/48 [00:04<01:31,  2.03s/it]



  8%|▊         | 4/48 [00:05<01:09,  1.58s/it]



 10%|█         | 5/48 [00:06<01:01,  1.43s/it]



 12%|█▎        | 6/48 [00:07<00:53,  1.27s/it]



 15%|█▍        | 7/48 [00:08<00:44,  1.09s/it]



 17%|█▋        | 8/48 [00:08<00:36,  1.11it/s]



 19%|█▉        | 9/48 [00:10<00:42,  1.09s/it]



 21%|██        | 10/48 [00:10<00:33,  1.12it/s]



 23%|██▎       | 11/48 [00:12<00:39,  1.07s/it]



 25%|██▌       | 12/48 [00:12<00:32,  1.11it/s]



 27%|██▋       | 13/48 [00:13<00:27,  1.27it/s]



 29%|██▉       | 14/48 [00:14<00:30,  1.13it/s]



 31%|███▏      | 15/48 [00:15<00:30,  1.09it/s]



 33%|███▎      | 16/48 [00:15<00:25,  1.26it/s]



 35%|███▌      | 17/48 [00:16<00:23,  1.29it/s]



 38%|███▊      | 18/48 [00:17<00:26,  1.13it/s]



 40%|███▉      | 19/48 [00:18<00:21,  1.33it/s]



 42%|████▏     | 20/48 [00:18<00:18,

...


Epoch 9/9
----------






  1%|          | 1/142 [00:05<12:26,  5.29s/it]



  1%|▏         | 2/142 [00:06<09:45,  4.18s/it]



  2%|▏         | 3/142 [00:08<07:49,  3.38s/it]



  3%|▎         | 4/142 [00:09<06:24,  2.79s/it]



  4%|▎         | 5/142 [00:10<05:12,  2.28s/it]



  4%|▍         | 6/142 [00:12<04:32,  2.00s/it]



  5%|▍         | 7/142 [00:13<04:08,  1.84s/it]



  6%|▌         | 8/142 [00:15<03:50,  1.72s/it]



  6%|▋         | 9/142 [00:16<03:22,  1.52s/it]



  7%|▋         | 10/142 [00:17<03:03,  1.39s/it]



  8%|▊         | 11/142 [00:18<02:49,  1.29s/it]



  8%|▊         | 12/142 [00:19<02:36,  1.21s/it]



  9%|▉         | 13/142 [00:20<02:28,  1.15s/it]



 10%|▉         | 14/142 [00:21<02:21,  1.11s/it]



 11%|█         | 15/142 [00:22<02:17,  1.08s/it]



 11%|█▏        | 16/142 [00:23<02:13,  1.06s/it]



 12%|█▏        | 17/142 [00:24<02:10,  1.04s/it]



 13%|█▎        | 18/142 [00:25<02:08,  1.04s/it]



 13%|█▎        | 19/142 [00:26<02:06,  1.03s/it]



 14%|█▍        | 

train Loss: 0.0050 Acc: 0.9873






  2%|▏         | 1/48 [00:06<04:57,  6.34s/it]



  4%|▍         | 2/48 [00:07<03:40,  4.79s/it]



  6%|▋         | 3/48 [00:09<02:59,  3.99s/it]



  8%|▊         | 4/48 [00:10<02:10,  2.97s/it]



 10%|█         | 5/48 [00:10<01:37,  2.28s/it]



 12%|█▎        | 6/48 [00:14<01:57,  2.79s/it]



 15%|█▍        | 7/48 [00:18<02:00,  2.94s/it]



 17%|█▋        | 8/48 [00:18<01:30,  2.26s/it]



 19%|█▉        | 9/48 [00:19<01:11,  1.83s/it]



 21%|██        | 10/48 [00:22<01:25,  2.26s/it]



 23%|██▎       | 11/48 [00:23<01:08,  1.84s/it]



 25%|██▌       | 12/48 [00:24<00:53,  1.49s/it]



 27%|██▋       | 13/48 [00:25<00:42,  1.20s/it]



 29%|██▉       | 14/48 [00:28<01:07,  2.00s/it]



 31%|███▏      | 15/48 [00:29<00:53,  1.61s/it]



 33%|███▎      | 16/48 [00:30<00:42,  1.32s/it]



 35%|███▌      | 17/48 [00:31<00:36,  1.18s/it]



 38%|███▊      | 18/48 [00:33<00:43,  1.43s/it]



 40%|███▉      | 19/48 [00:33<00:32,  1.12s/it]



 42%|████▏     | 20/48 [00:33<00:25,

val Loss: 0.0005 Acc: 1.0000

Training complete in 38m 13s
Best val Acc: 1.000000


In [91]:
torch.save(model_ft,"models/final")

NameError: name 'model_ft' is not defined

In [22]:
model = torch.load('models/final')
model.eval()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [25]:
image_datasets_t = datasets.ImageFolder("./test/birds",data_transforms)

RuntimeError: Found 0 files in subfolders of: ./test/birds
Supported extensions are: .jpg,.jpeg,.png,.ppm,.bmp,.pgm,.tif

In [134]:
dataloaders_t = torch.utils.data.DataLoader(image_datasets_t, batch_size=batch_size,
                                         shuffle=True, num_workers=4)
dataset_sizes_t = len(image_datasets_t)

In [135]:
for data in tqdm(dataloaders_t):
    # get the inputs
    inputs, labels = data
#     print(inputs)
    print(labels)
    for i in labels:
        print(class_names[i])

tensor([0, 4, 2, 3, 2, 0, 4, 0, 3, 0])
aircrafts_1
flowers_1
cars_1
dogs_1
cars_1
aircrafts_1
flowers_1
aircrafts_1
dogs_1
aircrafts_1
tensor([3, 0, 4, 4, 4, 0, 4, 0, 3, 3])
dogs_1
aircrafts_1
flowers_1
flowers_1
flowers_1
aircrafts_1
flowers_1
aircrafts_1
dogs_1
dogs_1
tensor([0, 0, 4, 3, 4, 0, 3, 2, 2, 3])
aircrafts_1
aircrafts_1
flowers_1
dogs_1
flowers_1
aircrafts_1
dogs_1
cars_1
cars_1
dogs_1
tensor([4, 3, 0, 2, 4, 0, 1, 2, 4, 4])
flowers_1
dogs_1
aircrafts_1
cars_1
flowers_1
aircrafts_1
birds_1
cars_1
flowers_1
flowers_1
tensor([3, 3, 0, 4, 3, 1, 3, 4, 2, 0])
dogs_1
dogs_1
aircrafts_1
flowers_1
dogs_1
birds_1
dogs_1
flowers_1
cars_1
aircrafts_1
tensor([0, 0, 0, 0, 3, 0, 4, 4, 3, 3])
aircrafts_1
aircrafts_1
aircrafts_1
aircrafts_1
dogs_1
aircrafts_1
flowers_1
flowers_1
dogs_1
dogs_1
tensor([1, 4, 4, 4, 0, 0, 2, 2, 0, 4])
birds_1
flowers_1
flowers_1
flowers_1
aircrafts_1
aircrafts_1
cars_1
cars_1
aircrafts_1
flowers_1
tensor([4, 0, 1, 0, 2, 2, 0, 3, 3, 0])
flowers_1
aircrafts_1
bir

In [136]:
for data in tqdm(dataloaders_t):
    # get the inputs
    inputs, labels = data
#         print(class_names[i])
    

In [137]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(list(filter(lambda p: p.requires_grad, model_conv.parameters())), lr=0.001, momentum=0.9)
running_loss=0.0
running_corrects=0
for data in tqdm(dataloaders_t):
    # get the inputs
    inputs, labels = data
#     for i in labels:
#         print('labels '+ class_names[i])

    if use_gpu:
        inputs = inputs.cuda()
        labels = labels.cuda()
    else:
        inputs, labels = inputs,labels

        optimizer.zero_grad()
#     with torch.set_grad_enabled(True):
    outputs = model(inputs)
    if type(outputs) == tuple:
        outputs, _ = outputs
    _,preds = torch.max(outputs,1)
    loss=criterion(outputs,labels.cuda())
    for i,j in zip(preds,labels):
        print('predicted '+ class_names[i])
        print('labels '+ class_names[j])
    
    # statistics
    running_loss += loss.data.item()
    running_corrects += torch.sum(preds == labels.data)
        
    epoch_loss = running_loss / dataset_sizes_t
    epoch_acc = running_corrects.double() / dataset_sizes_t
    #print(' Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

predicted dogs_1
labels dogs_1
predicted dogs_1
labels birds_1
predicted cars_1
labels cars_1
predicted flowers_1
labels flowers_1
predicted aircrafts_1
labels aircrafts_1
predicted flowers_1
labels flowers_1
predicted cars_1
labels cars_1
predicted birds_1
labels birds_1
predicted cars_1
labels cars_1
predicted flowers_1
labels flowers_1
predicted flowers_1
labels flowers_1
predicted dogs_1
labels dogs_1
predicted dogs_1
labels dogs_1
predicted dogs_1
labels dogs_1
predicted flowers_1
labels flowers_1
predicted birds_1
labels birds_1
predicted flowers_1
labels flowers_1
predicted birds_1
labels birds_1
predicted birds_1
labels birds_1
predicted aircrafts_1
labels aircrafts_1
predicted flowers_1
labels flowers_1
predicted cars_1
labels cars_1
predicted aircrafts_1
labels aircrafts_1
predicted cars_1
labels cars_1
predicted cars_1
labels cars_1
predicted dogs_1
labels dogs_1
predicted aircrafts_1
labels aircrafts_1
predicted flowers_1
labels flowers_1
predicted cars_1
labels cars_1
pred

predicted aircrafts_1
labels aircrafts_1
predicted aircrafts_1
labels aircrafts_1
predicted dogs_1
labels dogs_1
predicted dogs_1
labels dogs_1
predicted aircrafts_1
labels aircrafts_1
predicted aircrafts_1
labels aircrafts_1
predicted flowers_1
labels flowers_1
predicted birds_1
labels birds_1
predicted dogs_1
labels dogs_1
predicted aircrafts_1
labels aircrafts_1
predicted dogs_1
labels dogs_1
predicted cars_1
labels cars_1
predicted flowers_1
labels flowers_1
predicted aircrafts_1
labels aircrafts_1
predicted dogs_1
labels dogs_1
predicted aircrafts_1
labels aircrafts_1
predicted cars_1
labels cars_1
predicted aircrafts_1
labels aircrafts_1
predicted aircrafts_1
labels aircrafts_1
predicted aircrafts_1
labels aircrafts_1
predicted cars_1
labels cars_1
predicted dogs_1
labels dogs_1
predicted flowers_1
labels flowers_1
predicted flowers_1
labels flowers_1
predicted flowers_1
labels flowers_1
predicted aircrafts_1
labels aircrafts_1
predicted birds_1
labels birds_1
predicted cars_1
la

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(list(filter(lambda p: p.requires_grad, model_conv.parameters())), lr=0.001, momentum=0.9)
running_loss=0.0
running_corrects=0
for data in tqdm(dataloaders_t):
    # get the inputs
    inputs, labels = data
#     for i in labels:
#         print('labels '+ class_names[i])

    if use_gpu:
        inputs = inputs.cuda()
        labels = labels.cuda()
    else:
        inputs, labels = inputs,labels

        optimizer.zero_grad()
#     with torch.set_grad_enabled(True):
    outputs = model(inputs)
    if type(outputs) == tuple:
        outputs, _ = outputs
    _,preds = torch.max(outputs,1)
    loss=criterion(outputs,labels.cuda())
    for i,j in zip(preds,labels):
        print('predicted '+ class_names[i])
        print('labels '+ class_names[j])
    
    # statistics
    running_loss += loss.data.item()
    running_corrects += torch.sum(preds == labels.data)
        
    epoch_loss = running_loss / dataset_sizes_t
    epoch_acc = running_corrects.double() / dataset_sizes_t
    #print(' Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

In [138]:
print(' Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

 Loss: 0.0103 Acc: 0.9704


In [55]:
inputs

tensor([[[[-0.3725, -0.3569, -0.3176,  ..., -0.2392, -0.2627, -0.2784],
          [-0.3020, -0.3255, -0.2941,  ..., -0.0039,  0.0667,  0.0196],
          [-0.2863, -0.3176, -0.3020,  ...,  0.5686,  0.6314,  0.6549],
          ...,
          [ 0.1529,  0.1216,  0.1294,  ...,  0.4196,  0.3412,  0.4745],
          [ 0.1922,  0.1608,  0.1686,  ...,  0.4431,  0.3804,  0.4745],
          [ 0.1922,  0.1765,  0.2000,  ...,  0.4510,  0.3490,  0.4431]],

         [[-0.5059, -0.4980, -0.4824,  ..., -0.0745, -0.1294, -0.1294],
          [-0.4667, -0.4588, -0.4510,  ...,  0.1529,  0.1765,  0.1529],
          [-0.4588, -0.4510, -0.4588,  ...,  0.5843,  0.6549,  0.7098],
          ...,
          [-0.0667, -0.0667, -0.0667,  ...,  0.2706,  0.0980,  0.2078],
          [-0.0588, -0.0431, -0.0510,  ...,  0.2627,  0.1451,  0.2549],
          [-0.0353, -0.0275, -0.0196,  ...,  0.2627,  0.1137,  0.2157]],

         [[-0.6157, -0.6314, -0.6000,  ...,  0.0510, -0.0039, -0.0118],
          [-0.5529, -0.5843, -

In [54]:
inputs.size()

torch.Size([3, 3, 299, 299])

In [50]:
running_loss=0.0
running_corrects=0
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(list(filter(lambda p: p.requires_grad, model_conv.parameters())), lr=0.001, momentum=0.9)
if use_gpu:
    inputs = inputs.cuda()
    labels = labels.cuda()
else:
    inputs, labels = Variable(inputs), Variable(labels)

In [53]:
# zero the parameter gradients
optimizer.zero_grad()
# with torch.set_grad_enabled(True):
for i in inputs:
    outputs = model(inputs)
    if type(outputs) == tuple:
        outputs, _ = outputs
    _,preds = torch.max(outputs,1)
    loss=criterion(outputs,labels.cuda())
    print(outputs)
    # statistics

tensor([[-2.9345,  7.8348, -3.9780, -0.8100, -1.2928],
        [ 7.4846, -1.6983, -2.3476, -2.1119, -1.5335],
        [-2.3385, -2.3718,  6.8982, -0.5805, -1.8613]], device='cuda:0',
       grad_fn=<AddmmBackward>)
tensor([[-2.9345,  7.8348, -3.9780, -0.8100, -1.2928],
        [ 7.4846, -1.6983, -2.3476, -2.1119, -1.5335],
        [-2.3385, -2.3718,  6.8982, -0.5805, -1.8613]], device='cuda:0',
       grad_fn=<AddmmBackward>)
tensor([[-2.9345,  7.8348, -3.9780, -0.8100, -1.2928],
        [ 7.4846, -1.6983, -2.3476, -2.1119, -1.5335],
        [-2.3385, -2.3718,  6.8982, -0.5805, -1.8613]], device='cuda:0',
       grad_fn=<AddmmBackward>)


In [52]:
outputs

tensor([[-2.9345,  7.8348, -3.9780, -0.8100, -1.2928],
        [ 7.4846, -1.6983, -2.3476, -2.1119, -1.5335],
        [-2.3385, -2.3718,  6.8982, -0.5805, -1.8613]], device='cuda:0',
       grad_fn=<AddmmBackward>)

In [ ]:
# zero the parameter gradients
optimizer.zero_grad()
with torch.set_grad_enabled(True):
    for data in tqdm(dataloaders_t):
        inputs, labels = data
        outputs = model(inputs)
        ps = torch.exp(outputs)
        
#     if type(outputs) == tuple:
#         outputs, _ = outputs
#     _,preds = torch.max(outputs,1)
#     loss=criterion(outputs,labels.cuda())
#     # statistics

In [ ]:
np.shape(ps)

In [42]:
    running_loss += loss.data.item()
    running_corrects += torch.sum(preds == labels.data)

In [43]:
    epoch_loss = running_loss / dataset_sizes_t
    epoch_acc = running_corrects.double() / dataset_sizes_t
    print(' Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

 Loss: 0.0000 Acc: 0.0063


In [ ]:
os.mkdir('./test_result')
for i in range(len(preds)):
    


In [1]:
from skimage.io import imread_collection

#your path 
col_dir = './test/birds/*.jpg'

#creating a collection with the available images
col = imread_collection(col_dir)

In [2]:
print(col)

['./test/birds/American_Goldfinch_0011_153588827.jpg', './test/birds/American_Goldfinch_0018_143360018.jpg', './test/birds/American_Goldfinch_0019_2471592845.jpg', './test/birds/American_Goldfinch_0020_468543137.jpg', './test/birds/American_Goldfinch_0023_2098520281.jpg', './test/birds/California_Gull_0001_1507593116.jpg', './test/birds/California_Gull_0003_3033320042.jpg', './test/birds/California_Gull_0012_357599904.jpg', './test/birds/California_Gull_0020_2401678158.jpg', './test/birds/Frigatebird_0010_2632633868.jpg', './test/birds/Frigatebird_0020_1479238152.jpg', './test/birds/Glaucous_winged_Gull_0001_2809414777.jpg', './test/birds/Glaucous_winged_Gull_0002_2314583028.jpg', './test/birds/Glaucous_winged_Gull_0020_355948038.jpg', './test/birds/Glaucous_winged_Gull_0021_2577916185.jpg', './test/birds/Glaucous_winged_Gull_0035_409311135.jpg', './test/birds/Heermann_Gull_0002_191127454.jpg', './test/birds/Heermann_Gull_0005_2787687738.jpg', './test/birds/Heermann_Gull_0019_169059513

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(list(filter(lambda p: p.requires_grad, model_conv.parameters())), lr=0.001, momentum=0.9)
running_loss=0.0
running_corrects=0
for data in tqdm(dataloaders_t):
    # get the inputs
    inputs, labels = data
#     for i in labels:
#         print('labels '+ class_names[i])

    if use_gpu:
        inputs = inputs.cuda()
        labels = labels.cuda()
    else:
        inputs, labels = inputs,labels

        optimizer.zero_grad()
#     with torch.set_grad_enabled(True):
    outputs = model(inputs)
    if type(outputs) == tuple:
        outputs, _ = outputs
    _,preds = torch.max(outputs,1)
    loss=criterion(outputs,labels.cuda())
    for i,j in zip(preds,labels):
        print('predicted '+ class_names[i])
        print('labels '+ class_names[j])
    
    # statistics
    running_loss += loss.data.item()
    running_corrects += torch.sum(preds == labels.data)
        
    epoch_loss = running_loss / dataset_sizes_t
    epoch_acc = running_corrects.double() / dataset_sizes_t
    #print(' Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

In [ ]:
# Load image
    real = Image.open('./real_A.jpg')
    preprocess = transforms.Compose([
        transforms.Scale(opt.loadSize),
        transforms.RandomCrop(opt.fineSize),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5),
                             (0.5, 0.5, 0.5)),
    ])  

    # Load input
    input_A = preprocess(real).unsqueeze_(0)
    model.input_A.resize_(input_A.size()).copy_(input_A)
    # Forward (model.real_A) through G and produce output (model.fake_B)
    model.test()

    # Convert image to numpy array
    fake = util.tensor2im(model.fake_B.data)
    # Save image
    util.save_image(fake, './fake_B.png')

In [ ]:
def predict_image(image_path):
    print("Prediction in progress")
    image = Image.open(image_path)

    # Define transformations for the image, should (note that imagenet models are trained with image size 224)
    transformation = transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

    ])

    # Preprocess the image
    image_tensor = transformation(image).float()

    # Add an extra batch dimension since pytorch treats all images as batches
    image_tensor = image_tensor.unsqueeze_(0)

    if torch.cuda.is_available():
        image_tensor.cuda()

    # Turn the input into a Variable
    input = Variable(image_tensor)

    # Predict the class of the image
    output = model(input)

    index = output.data.numpy().argmax()

    return index

In [ ]:
if __name__ == "__main__":

    imagefile = "image.png"

    imagepath = os.path.join(os.getcwd(), imagefile)
    # Donwload image if it doesn't exist
    if not os.path.exists(imagepath):
        data = requests.get(
            "https://github.com/OlafenwaMoses/ImageAI/raw/master/images/3.jpg", stream=True)

        with open(imagepath, "wb") as file:
            shutil.copyfileobj(data.raw, file)

        del data

    index_file = "class_index_map.json"

    indexpath = os.path.join(os.getcwd(), index_file)
    # Donwload class index if it doesn't exist
    if not os.path.exists(indexpath):
        data = requests.get('https://github.com/OlafenwaMoses/ImageAI/raw/master/imagenet_class_index.json')

        with open(indexpath, "w", encoding="utf-8") as file:
            file.write(data.text)

    class_map = json.load(open(indexpath))

    # run prediction function annd obtain prediccted class index
    index = predict_image(imagepath)

    prediction = class_map[str(index)][1]

    print("Predicted Class ", prediction)